In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve 
from sklearn.metrics import make_scorer 

pd.options.display.max_rows = None

df = pd.read_csv("AB_NYC_2019.csv")

df.head(10)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0
5,5099,Large Cozy 1 BR Apartment In Midtown East,7322,Chris,Manhattan,Murray Hill,40.74767,-73.97500,Entire home/apt,200,3,74,2019-06-22,0.59,1,129
6,5121,BlissArtsSpace!,7356,Garon,Brooklyn,Bedford-Stuyvesant,40.68688,-73.95596,Private room,60,45,49,2017-10-05,0.40,1,0
7,5178,Large Furnished Room Near B'way,8967,Shunichi,Manhattan,Hell's Kitchen,40.76489,-73.98493,Private room,79,2,430,2019-06-24,3.47,1,220
8,5203,Cozy Clean Guest Room - Family Apt,7490,MaryEllen,Manhattan,Upper West Side,40.80178,-73.96723,Private room,79,2,118,2017-07-21,0.99,1,0
9,5238,Cute & Cozy Lower East Side 1 bdrm,7549,Ben,Manhattan,Chinatown,40.71344,-73.99037,Entire home/apt,150,1,160,2019-06-09,1.33,4,188


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              48895 non-null  int64  
 1   name                            48879 non-null  object 
 2   host_id                         48895 non-null  int64  
 3   host_name                       48874 non-null  object 
 4   neighbourhood_group             48895 non-null  object 
 5   neighbourhood                   48895 non-null  object 
 6   latitude                        48895 non-null  float64
 7   longitude                       48895 non-null  float64
 8   room_type                       48895 non-null  object 
 9   price                           48895 non-null  int64  
 10  minimum_nights                  48895 non-null  int64  
 11  number_of_reviews               48895 non-null  int64  
 12  last_review                     

In [3]:
df = df[df.price > 0]
df['reviews_per_month'].fillna(0, inplace=True)

In [4]:
np.unique(df.neighbourhood_group, return_counts=True)

(array(['Bronx', 'Brooklyn', 'Manhattan', 'Queens', 'Staten Island'],
       dtype=object),
 array([ 1090, 20095, 21660,  5666,   373], dtype=int64))

In [5]:

df = df[df['neighbourhood_group'] == 'Staten Island']

df

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
169,42882,New York room with a view,185978,Newyorkroomwithaview,Staten Island,St. George,40.64524,-74.08088,Private room,70,2,166,2019-06-13,1.66,1,312
249,62452,A SpeciaL!! Private Room in NY,303939,Lissette,Staten Island,Tompkinsville,40.63536,-74.08537,Private room,36,2,193,2019-06-25,1.85,6,360
250,62461,B NYC Staten Alternative...,303939,Lissette,Staten Island,Tompkinsville,40.63627,-74.08543,Private room,37,2,147,2019-06-10,1.44,6,0
251,62787,C Private Room By The Ferry,303939,Lissette,Staten Island,Tompkinsville,40.63518,-74.08546,Private room,37,2,177,2019-07-02,1.71,6,320
256,63320,D Private Che@p Room 2 Explore NYC,303939,Lissette,Staten Island,Tompkinsville,40.63481,-74.08519,Private room,36,2,333,2019-07-02,3.19,6,340
571,214917,New Clean Spacious Bed & Breakfast,1109658,Carmel,Staten Island,Emerson Hill,40.60742,-74.14388,Private room,80,2,2,2018-02-08,0.05,1,158
598,225976,Sunny cozy multileveled apartment!,1177947,Marina,Staten Island,Shore Acres,40.61077,-74.06824,Entire home/apt,75,6,76,2019-06-13,0.80,1,278
702,258838,"Oceanview,close to Manhattan",1360198,Marina,Staten Island,Arrochar,40.59251,-74.06479,Entire home/apt,250,2,21,2019-05-22,0.26,4,333
703,258876,"Affordable rooms,all transportation",1360198,Marina,Staten Island,Arrochar,40.59101,-74.06685,Private room,50,7,0,NaN,0.00,4,189
704,259946,"Budget stay, near transportation",1360198,Marina,Staten Island,Arrochar,40.59262,-74.06659,Entire home/apt,125,2,8,2019-05-19,0.09,4,353


In [6]:
columnNames = ['minimum_nights',
              'reviews_per_month',
              'calculated_host_listings_count',
              'number_of_reviews', 'price']

data = df[columnNames]

columnNames.remove('price')

x = data[columnNames]
y = df.price

data

,minimum_nights,reviews_per_month,calculated_host_listings_count,number_of_reviews,price
169,2,1.66,1,166,70
249,2,1.85,6,193,36
250,2,1.44,6,147,37
251,2,1.71,6,177,37
256,2,3.19,6,333,36
571,2,0.05,1,2,80
598,6,0.80,1,76,75
702,2,0.26,4,21,250
703,7,0.00,4,0,50
704,2,0.09,4,8,125


In [7]:
x.info()

<class 'pandas.core.frame.DataFrame'>
Index: 373 entries, 169 to 48799
Data columns (total 4 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   minimum_nights                  373 non-null    int64  
 1   reviews_per_month               373 non-null    float64
 2   calculated_host_listings_count  373 non-null    int64  
 3   number_of_reviews               373 non-null    int64  
dtypes: float64(1), int64(3)
memory usage: 14.6 KB


In [8]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [12]:
from sklearn.linear_model import LinearRegression

model = LinearRegression().fit(x_train, y_train)
y_pred = model.predict(x_test)

resultData = x_test.join(y_test)
resultData['predicted_price'] = y_pred

score = model.score(x, y)
print('model score:', score)
resultData

model score: 0.020769386994282102


,minimum_nights,reviews_per_month,calculated_host_listings_count,number_of_reviews,price,predicted_price
26191,1,3.34,3,74,52,70.633393
46459,2,0.00,1,0,49,138.979927
22263,2,0.43,3,12,89,110.034382
36987,3,2.68,1,19,90,108.131046
47316,1,0.00,1,0,42,139.054081
36256,1,2.43,1,23,89,110.304660
26784,2,1.41,1,29,115,120.005304
1434,2,0.33,2,23,71,120.816054
28890,2,0.53,2,10,75,120.631197
5478,2,2.42,1,139,180,93.344643


In [ ]:
plt.scatter(x.calculated_host_listings_count, y)